# Udacity Self-Driving Car Nanodegree Advanced Lane Finding Module

### Computer Vision for Mapping of the environment

Robotics can be broken down into a three step cycle:
* Step 1: Sensing our environment accuratly.
* Step 2: Decide what to do based on that data.
* Step 3: Perform an action based on that decision. And Repeat

<div class="img-with-text">
    <img src="img/robotics_cycle.png" alt="sometext" style="width: 200px;"/>
    <p style="text-align: center;"><b>Fig. 1:</b><em>(Three basic steps of robotics)</em></p>
</div>

Computer Vision deals with perception which is the first step in robotics. According to Sebastian Thrun, *80% of the challenge of building self-driving cars is perception*. So, Computer vision is the art and science of percieving the environment around us using images. It can help us detect lane lines, obsticles, vehicles, pedestrians e.t.c. This gives us a map of the environment around us. 

Cameras and Computer Vision are a great techniques with it's own pros and cons. Cameras are cheaper that more sophisticated technologies such as Lasers and Radars. Laser and Radars give us a complete 3D map of our environment while cameras only give us 2D images. But the images from cameras have a greater resolution compared to that of Laser and Radar. Also, Camaera are very cheap compared to Laser and Radar technology.

In the [**1st Project**](https://github.com/muddassir235/Lane-Line-Detection-SDC-Udacity) of the Nanodegree we made a basic program that could detect straight lane lines along the road. In this module we will apply more advanced computer vision techniques in order to make a higly robust lane finding algorithm which can detect;
* Curvy lanes
* Can detect lane lines correctly even if there are shadows
* Ignores changes of the pavment color e.t.c.



<div class="img-with-text">
    <img src="https://profalbrecht.files.wordpress.com/2009/07/road01.jpg" alt="Curvy Road" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 2:</b><em> (Curvy Road) Credits: [profalbrecht.wordpress.com](https://profalbrecht.wordpress.com)</em></p>
</div>

This will help us in finding the distance of the vehicles from the lane lines and the curvature of the road ahead. This information is higly useful for making control decisions in a self-driving car.

After following the Advanced Lane Finding module we have the Vehicle Detection and Tracking module. The vehicles detection and tracking module will let us know the exact position of other vehicles relative to us. This information is needed when making decision about changing lanes, acceleration and decelleration. 

By combining the _Advanced Lane Finding_ module and the _Vehicle Detection and Tracking_ module we will make a program that tells us exactly where the cars is on the road and the vehicles in it's way.

## Advanced Lane Finding

In this module we need to measure some of the quantities need to control the car. For example, in order to steer the car we need to know the curvature of the road in front of us. In order measure the curvature of the road we need to apply a transform to the images which gives us a top view of the road in front of us.

<div class="img-with-text">
    <img src="img/transform.png" alt="Curvy Road" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 3:</b><em> (Tranform from front view to top view Credits: Udacity)</em></p>
</div>

#### Distortions in Images

In order to get this perspective tranformation right we need to remove distortion from the images which are caused due to camera lens. These distortions skew the objects at the edges of the images and will lead to in accurate results if we don't remove them before hand. Distortion occur when a camera tranforms 3D objects in the real world into 2D images. The distortion change the shapes and sizes of the objects in the images. They can also cause an object to appear closer, or farther away than it actually is.

The our case these distortions may lead to incurrate calculations of the curvature of the road, which in turn will lead to higly undesired steering angles. So, it is crucial that we remove these distortions first.

<div class="img-with-text">
    <img src="img/distortions.png" alt="Distortion" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 4:</b><em> (Distortions due to camera lenses. Credits: Udacity)</em></p>
</div>

Before we jump to writing code in order to correct for distortions in the images, we need to discuss the theory behind these distortions so that we can model them properly. 

The simplest and oldest model of a camera is the pin hole camera. A pinhole camera is similar to our eyes. Rays of light starting from the object pass through a very small and are focused on back of our eyes, or in case of a camera a screen or detector. During this process, rays of light from the top of the object stick and form the bottom part of it's image and vise versa. Similar is the case for the left and right parts of the object. So, after this we get an upside down image of the object on a screen.

<div class="img-with-text">
    <img src="img/pinhole_camera.png" alt="Pinhole camera" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 5:</b><em> (Pihole camera and human eye. Credits: Udacity)</em></p>
</div>

A camera tranforms an object in 3D to an image in 2D. This is can be modelled by a matrix tranformation with a matrix which we call the camera matrix. 

<div class="img-with-text">
    <img src="img/3dto2d.png" alt="3D to 2D" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 6:</b><em> (Camera Tranform. Credits: Udacity)</em></p>
</div>

But nowadays cameras don't use pinholes, they use lens, and lens can cause rays of light to bend unusually, leading to distortions in the image that is formed. This type of distortion that is caused by the unusal bending of rays is called **Radial Distortion**

There is another type of distortion known as **Tangential Distortion**. This type of distortion leads to the place of the image bieng distorted. It is caused when the plane containing the lens is not parallel to that containing our sensor or screen. 

<div class="img-with-text">
    <img src="img/tangential_distortion.png" alt="Tangential Distortion" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 7:</b><em> (Tangential distortion. Credits: Udacity)</em></p>
</div>

All of these distortion can modelled using five coefficients known as **Distortion Coefficients**. The values of these coefficients model the amount of Radial and Tangential distortions. These coefficients are unique for each camera, and if we know them we can undistort all of the images from that camera.

These coefficients are **(k1, k2, p1, p2, and k3)**. **k1, k2** and **k3** model the Radial Distortion in an image, and **p1 and p2** model the Tangential Distortion in it. To correct the appearance of a radially distorted image we can use a correction formula. 

Suppose *(x,y)* is a point in the distorted image, in order correct for the distortion OpenCV calculates the radial distance between a point in an undistorted image *(x_corrected, y_corrected)* and the center of distortion *(x_c, y_c)* (*This is the place or point which is at it's original postion*), usually this will be at the center of the image. Let's call this distance **r**.

<div class="img-with-text">
    <img src="https://d17h27t6h515a5.cloudfront.net/topher/2016/December/5840ae19_screen-shot-2016-12-01-at-3.10.19-pm/screen-shot-2016-12-01-at-3.10.19-pm.png" alt="Radial Distance" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 8:</b><em> (The radial distance from a corrected point to the distortion center <b>r</b>. Credits: Udacity)</em></p>
</div>

**Note:** **k3** *is used for modelling major distortion, and in case of minor distortion it usually has a value at or close to zero. In OpenCV one can ignore this coefficient, but we will be using this coefficient in all our models so that it applying in wider range of scinareos and correct for both major and minor distortions*

The formula for correction of **Radial Distortion** is as follows

```python
x_corrected = x * (1 + k1 * r**2 + k2 * r**4 + k3 * r**6)
y_corrected = y * (1 + k1 * r**2 + k2 * r**4 + k3 * r**6)
```

The formuala for correction of **Tangential Distortion** is as follows

```python 
x_corrected = x + (2*p1*x*y + p2*(r**2 + 2*x**2))
y_corrected = y + (p1*(r**2 + 2*y**2) + 2*p2*x*y)
```

As we know that distoration can change the shapes, sizes and distances between objects. In order correct for these distortions we need to compare images taken by a specfic camera of an object to it's original form so we create a map between distorted and undistored images. In order to do that we need a good control object. A **Chess Board** is a great example of such an object. It has a regular and contrasting pattern which is easily detectable. So using this pattern we can automatically calibrate for our camera and find a mapping from distorted to undistorted points.

<div class="img-with-text">
    <img src="img/map_to_undistorted.png" alt="Map to undistorted" style="width: 500px;" vspace=20/>
    <p style="text-align: center;"><b>Fig. 9:</b><em> (Find a map from distorted points to undistorted ones, for a chess board. Credits: Udacity)</em></p>
</div>